### 掛載雲端硬碟

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1.將資料向量化

In [3]:
import csv
import numpy as np
import tensorflow as tf
# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "/content/drive/MyDrive/creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)


HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


## 2.準備驗證資料

In [4]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))


Number of training samples: 227846
Number of validation samples: 56961


## 3.分析不平衡資料 (少數資料)

In [5]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]


Number of positive samples in training data: 417 (0.18% of total)


## 4.統計資料

In [6]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std


## 5.建立模型

In [13]:
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 256)               7936      
                                                                 
 dense_9 (Dense)             (None, 256)               65792     
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_10 (Dense)            (None, 256)               65792     
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_11 (Dense)            (None, 1)                 257       
                                                                 
Total params: 139,777
Trainable params: 139,777
Non-tr

### 6.評估指標:F1Score、Recall、Precision

In [14]:
from sklearn.metrics import classification_report, accuracy_score
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, val_data):
        super(Metrics, self).__init__()
        self.x_val = val_data[0]
        self.y_val = val_data[1]

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.x_val, batch_size=64)
        y_pred = np.argmax(y_pred, axis=1)
        y_val = np.argmax(self.y_val, axis=1)
        result = classification_report(y_val, y_pred)      

        print(result)
        return

## 7.使用class_weight訓練模型

In [16]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

# callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=128,
    epochs=30,
    verbose=2,
    callbacks=Metrics(val_data=(val_features, val_targets)),
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)


Epoch 1/30
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56961

    accuracy                           1.00     56961
   macro avg       1.00      1.00      1.00     56961
weighted avg       1.00      1.00      1.00     56961

1781/1781 - 22s - loss: 3.8018e-06 - fn: 18.0000 - fp: 36899.0000 - tn: 190530.0000 - tp: 399.0000 - precision: 0.0107 - recall: 0.9568 - val_loss: 0.0171 - val_fn: 12.0000 - val_fp: 318.0000 - val_tn: 56568.0000 - val_tp: 63.0000 - val_precision: 0.1654 - val_recall: 0.8400 - 22s/epoch - 12ms/step
Epoch 2/30
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56961

    accuracy                           1.00     56961
   macro avg       1.00      1.00      1.00     56961
weighted avg       1.00      1.00      1.00     56961

1781/1781 - 20s - loss: 3.8545e-06 - fn: 17.0000 - fp: 37271.0000 - tn: 190158.0000 - tp: 400.0000 - precision: 0.0106 - recall: 0.9